# Разработка рекомендательной модели машинного обучения

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.


## 1. Исследование данных

Для начала подтянем библиотеки :)

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split

import pandas as pd

In [2]:
users_data = pd.read_csv("/datasets/users_behavior.csv")
users_data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


5 столбцов, длительность звонков, количество минут, сообщений, мегабайт (потраченных за месяц) и принадлежность к тарифу "Смарт" или "Ультра".

## 2. Подготовка данных

60% данных - обучающая выборка, 20% - валидационная, 20% - тестовая. 

In [3]:
users_train, users_valid_test = train_test_split(users_data, test_size=0.4, random_state=12345)
users_valid, users_test = train_test_split(users_valid_test, test_size=0.5, random_state=12345)
print(users_train.shape)
print(users_valid.shape)
print(users_test.shape)

(1928, 5)
(643, 5)
(643, 5)


60-20-20%. Разделение готово. Теперь нарежем полученные выборки на датасеты с параметрами и целевым параметром (принадлежностью к тому или иному тарифному плану - ведь нас интересует именно возможность такой классификации). 

In [4]:
features_train = users_train.drop(['is_ultra'], axis = 1)
target_train = users_train['is_ultra']

features_valid = users_valid.drop(['is_ultra'], axis = 1)
target_valid = users_valid['is_ultra']

features_test = users_test.drop(['is_ultra'], axis = 1)
target_test = users_test['is_ultra']

## 3. Обучение и валидация моделей

Начнем с классифицирующего древа. Проверим влияние глубины древа (как гиперпараметра) на accuracy на валидационной выборке. Сделаем шаг в 1 единицу (можем себе позволить!). Диапазон варьирования - от 2 до 50 уровней. 

In [5]:
max_accuracy_tree = 0
max_tree_depth = 0
for i in range (2, 51, 1):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth = i)
    model_tree.fit(features_train, target_train)
    predictions_tree = model_tree.predict(features_valid)
    accuracy_tree = accuracy_score(target_valid, predictions_tree)
    if max_accuracy_tree < accuracy_tree:
        max_accuracy_tree = accuracy_tree
        max_tree_depth = i
              
print("Наилучшее значение accuracy, равное", max_accuracy_tree, "зафиксировано для глубины дерева, равной", max_tree_depth)

Наилучшее значение accuracy, равное 0.7853810264385692 зафиксировано для глубины дерева, равной 3


Оказывается не нужно было "глубоко" смотреть - наилучший результат получен для древа глубиной 3. Переходим к изучению модели случайных деревьев.

In [6]:
max_accuracy_forest = 0
max_forest_estimators_number = 0
for i in range (2, 51, 1):
    model_forest = RandomForestClassifier(random_state=12345, n_estimators = i)
    model_forest.fit(features_train, target_train)
    predictions_forest = model_forest.predict(features_valid)
    accuracy_forest = accuracy_score(target_valid, predictions_forest)
    if max_accuracy_forest < accuracy_forest:
        max_accuracy_forest = accuracy_forest
        max_forest_estimators_number = i
              
print("Наилучшее значение accuracy, равное", max_accuracy_forest,
      "зафиксировано для числа деревьев, равного", max_forest_estimators_number)

Наилучшее значение accuracy, равное 0.7947122861586314 зафиксировано для числа деревьев, равного 23


Налицо куда большая длительность обучения и валидации модели по данному алгоритму. Наилучший результат (надо сказать, не намного лучший чем у решающего древа), получен для 23 случайных деревьев. Переходим к логистической регрессии.

In [7]:
model_log_reg = LogisticRegression()
model_log_reg.fit(features_train, target_train)
predictions_log_reg = model_log_reg.predict(features_valid)
accuracy_log_reg = accuracy_score(target_valid, predictions_log_reg)
print("Для модели на основе алгоритма логистической регрессии, значение accuracy равно", accuracy_log_reg)

Для модели на основе алгоритма логистической регрессии, значение accuracy равно 0.7589424572317263


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Минимальное значение accuracy получено для модели, обученной по алгоритму логистической регрессии. Целесообразность его дальнейшего использования сомнительна.

Таким образом, у нас есть победитель - модель решающего древа с максимальной глубиной = 3. Для наших целей модели, обученной по данному алгоритму более чем достаточно, т.к. она сочетает в себе достаточно высокую accuracy (на имеющихся данных) и высокую скорость, в отличие от алгоритма с 23 (!) деревьями и логистической регрессией. 

Для "леса" accuracy не намного выше (~0.01), а длительность намного выше, что, в реальных условиях может быть куда губительней, нежели потеря 1% accuracy предсказания.


Модель, обученная по алгоритму логистической регрессии, на валидационных данных позволяет получить accuracy чуть большую 0.75 при скорости, сопоставимой с решающим древом. Как следствие, данный вариант модели выпадает из "конкуренции".

## 4. Тестирование модели

In [8]:
features_full = pd.concat([features_train,features_valid])
target_full = pd.concat([target_train,target_valid])

model_tree_final = DecisionTreeClassifier(random_state=12345, max_depth = 3)
model_tree_final.fit(features_full, target_full)
predictions = model_tree_final.predict(features_test)
accuracy = accuracy_score(target_test, predictions)
accuracy

0.776049766718507

На тестовой выборке мы получили accuracy более 0.75 - почти 0.78. Рубеж взят :)